# Mines, Part 2

You can get information about a specific mine by using its Mine ID.

**Try searching using the Mine ID `3503598`**.

## Preparation: Knowing your tags

These questions are the same for every data set, and might not work exactly for yours.

### What is the tag and class name for the mine operator name?

In [1]:
#<td> <font style="FONT-SIZE:.80em; color:#000080"><strong>Operator:</strong> </font></td>

### What is the tag and class name for the current controller?

In [2]:
# <td width="40%"> <font style="FONT-SIZE:.80em; color:#000080"><b><!-- DNT -->S-2 Contractors Inc<!-- /DNT --></b></font></td>

### What is the tag and class name for the operator history area?

In [3]:
# <tr> <td colspan="3" valign="top"><div class="drsoprhistory">Operator History for Mine ID:  <b>3503598</b></div>  </td></tr>

### What is the tag and class name for the mine's address?

In [4]:
# <td width="40%"><font style="FONT-SIZE:.80em; color:#000080">6860 Anderson Rd.<br> Aurora, OR&nbsp;&nbsp;97002</font></td>

## Setup: Import what you'll need to scrape the page

Use `requests`, not `urllib`.

In [5]:
import requests
from bs4 import BeautifulSoup

## Scrape this page

Scrape this page, displaying the

- The operator
- The current address
- The current controller

**You should know how to do `.post` requests by now.**

In [6]:
data = {
'MineId':'3503598',
'x':'0',
'y':'0'    
}

url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
response_specific_mine = requests.post(url_specific_mine, data = data)

In [7]:
doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
doc_specific_mine

 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script src="/2010redesign/Scripts/federated-analytics.js" type="text/javascript"></script>
<script src="/2010redesign/Scripts/AC_RunActiveContent.js" type="text/javascript"></script>
<link href="/2010Redesign/includes/Print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="/2010Redesign/Includes/MSHAwebnew.css" media="screen" rel="stylesheet" type="text/css">
<link href="/2010Redesign/includes/style-screen.css" media="screen" rel="stylesheet" type="text/css"/>
</link></head>
<body>
<link href="https://webapps.dol.gov/FSPublic/Content/W_Helpful.css" rel="stylesheet" type="text/css"/>
<script src="https://ajax.g

In [8]:
doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})

[<font style="FONT-SIZE:.80em; color:#000080"><strong>Mine ID:</strong></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b>
 <input maxlength="7" name="MineId" size="10" type="hidden" value="3503598"/>3503598</b> </font>,
 <font style="FONT-SIZE:.80em; color:#000080"><strong>Operator:</strong> </font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b><!-- DNT -->Newberg Rock &amp; Dirt<!-- /DNT --></b> </font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b>Opr. Begin Date:</b></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b>12/6/2012</b></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><strong>Mine Name:</strong></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><strong><!-- DNT -->Newberg Rock &amp; Dirt<!-- /DNT --></strong></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b>Current Controller:</b></font>,
 <font style="FONT-SIZE:.80em; color:#000080"><b><!-- DNT -->
 S-2 Contractors Inc<!-- /DNT --></b></font>,
 <font style="FONT-SIZE:.80em; color:#0

In [9]:
operator = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[3].b
operator = operator.text
operator

'Newberg Rock & Dirt'

In [10]:
address = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[-1]
address = address.text.strip()
address = address.replace("\r\n", ", ")
address = address.replace("\xa0\xa0",", ")
address

'6860 Anderson Rd., Aurora, OR, 97002'

In [11]:
controller = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[9]
controller = controller.text.strip()
controller

'S-2 Contractors Inc'

## Getting information on many mines

### Reading in our source

Using pandas, read in `mines-subset.csv`.

In [12]:
import pandas as pd
mines_subset = pd.read_csv('mines-subset.csv')
mines_subset.shape

(4, 1)

In [13]:
mines_subset.dtypes

id    int64
dtype: object

In [14]:
mines_subset

,id
0,2501216
1,3200965
2,2901371
3,2901544


## Scrape every single row, storing the current controller and mine operator in new columns.

You probably want to open up the Jupyter Notebook that's about `.apply`.

In [15]:
def operator_name(row):
    data = {
        'MineId': row['id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    operator = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[3].b
    operator = operator.text
    return operator

mines_subset.apply(operator_name, axis=1)

mines_subset['operator_name'] = mines_subset.apply(operator_name, axis=1)
mines_subset

,id,operator_name
0,2501216,Iske Dirt Sand & Gravel
1,3200965,J M Lynn Dirtwork
2,2901371,Jake Diel Dirt & Paving Inc
3,2901544,Jake Diel Dirt & Paving Inc


In [16]:
def mine_address(row):
    data = {
        'MineId': row['id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    address = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[-1]
    address = address.text.strip()
    address = address.replace("\r\n", ", ")
    address = address.replace("\xa0\xa0",", ")
    return address

mines_subset.apply(mine_address, axis=1)

mines_subset['mine_address'] = mines_subset.apply(mine_address, axis=1)
mines_subset

,id,operator_name,mine_address
0,2501216,Iske Dirt Sand & Gravel,"24617 W Center Rd, Waterloo, NE, 68069"
1,3200965,J M Lynn Dirtwork,"485 Helene St, Palermo, ND, 58769"
2,2901371,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045"
3,2901544,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045"


In [17]:
def main_controller(row):
    data = {
        'MineId': row['id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    controller = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})[9]
    controller = controller.text.strip()
    controller
    return controller

mines_subset['main_controller'] = mines_subset.apply(main_controller, axis=1)
mines_subset

,id,operator_name,mine_address,main_controller
0,2501216,Iske Dirt Sand & Gravel,"24617 W Center Rd, Waterloo, NE, 68069",David A Iske
1,3200965,J M Lynn Dirtwork,"485 Helene St, Palermo, ND, 58769",John Lynn
2,2901371,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045",Lawson Warner
3,2901544,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045",Lawson Warner


### Save your dataframe

In [18]:
mines_subset.to_csv('mines-subset_expanded.csv', index = False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [20]:
pd.read_csv('mines-subset_expanded.csv')

,id,operator_name,mine_address,main_controller
0,2501216,Iske Dirt Sand & Gravel,"24617 W Center Rd, Waterloo, NE, 68069",David A Iske
1,3200965,J M Lynn Dirtwork,"485 Helene St, Palermo, ND, 58769",John Lynn
2,2901371,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045",Lawson Warner
3,2901544,Jake Diel Dirt & Paving Inc,"E Hwy 60, HEREFORD, TX, 79045",Lawson Warner


## Repeat this process for the entire `mines.csv` file

In [21]:
mines_csv = pd.read_csv('mines.csv', converters = {'id':str})
mines_csv.head()

,id
0,3503598
1,0502030
2,4801789
3,4201449
4,4201450


In [22]:
mines_csv = mines_csv.rename(index=str, columns={'id':'new_id'})
mines_csv.head()

,new_id
0,3503598
1,0502030
2,4801789
3,4201449
4,4201450


In [23]:
def operators_name(row):
    data = {
        'MineId': row['new_id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    font_tds = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})
    operators = font_tds[3].b
    operators = operators.text
    return operators

mines_csv['operator_name'] = mines_csv.apply(operators_name, axis=1)
mines_csv.head()

,new_id,operator_name
0,3503598,Newberg Rock & Dirt
1,0502030,Allied Dirt Moving Company
2,4801789,AM Dirtworks & Aggregate Sales
3,4201449,Atlas-Dirty Devil Mining
4,4201450,Atlas-Dirty Devil Mining


In [24]:
def mine_addresses(row):
    data = {
        'MineId': row['new_id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    font_tds = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})
    addresses = font_tds[23].text.strip()
    addresses = addresses.replace("\r\n", ", ")
    addresses = addresses.replace("\xa0\xa0",", ")
    return addresses

mines_csv['mine_address'] = mines_csv.apply(mine_addresses, axis=1)
mines_csv.head()

,new_id,operator_name,mine_address
0,3503598,Newberg Rock & Dirt,"6860 Anderson Rd., Aurora, OR, 97002"
1,0502030,Allied Dirt Moving Company,"2360 West 2nd Ave, DENVER, CO, 80010"
2,4801789,AM Dirtworks & Aggregate Sales,"120 Dally Ln, Buffalo, WY, 82834"
3,4201449,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525"
4,4201450,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525"


In [25]:
def main_controllers(row):
    data = {
        'MineId': row['new_id'],
        'x':'0',
        'y':'0'    
        }
    url_specific_mine = "https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp"
    response_specific_mine = requests.post(url_specific_mine, data = data)
    doc_specific_mine = BeautifulSoup(response_specific_mine.text, "html.parser")
    font_tds = doc_specific_mine.find_all('font', attrs = {'style':'FONT-SIZE:.80em; color:#000080'})
    controllers = font_tds[9].b
    controllers = controllers.text.strip()
    return controllers

mines_csv['main_controller'] = mines_csv.apply(main_controllers, axis=1)
mines_csv.head()

,new_id,operator_name,mine_address,main_controller
0,3503598,Newberg Rock & Dirt,"6860 Anderson Rd., Aurora, OR, 97002",S-2 Contractors Inc
1,0502030,Allied Dirt Moving Company,"2360 West 2nd Ave, DENVER, CO, 80010",Allied Dirt Moving Company
2,4801789,AM Dirtworks & Aggregate Sales,"120 Dally Ln, Buffalo, WY, 82834",Matt Mitchell
3,4201449,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co
4,4201450,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co


In [26]:
mines_csv.head()

,new_id,operator_name,mine_address,main_controller
0,3503598,Newberg Rock & Dirt,"6860 Anderson Rd., Aurora, OR, 97002",S-2 Contractors Inc
1,0502030,Allied Dirt Moving Company,"2360 West 2nd Ave, DENVER, CO, 80010",Allied Dirt Moving Company
2,4801789,AM Dirtworks & Aggregate Sales,"120 Dally Ln, Buffalo, WY, 82834",Matt Mitchell
3,4201449,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co
4,4201450,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co


In [27]:
mines_csv = mines_csv.rename(index=str, columns={'new_id':'id'})
mines_csv.head()

,id,operator_name,mine_address,main_controller
0,3503598,Newberg Rock & Dirt,"6860 Anderson Rd., Aurora, OR, 97002",S-2 Contractors Inc
1,0502030,Allied Dirt Moving Company,"2360 West 2nd Ave, DENVER, CO, 80010",Allied Dirt Moving Company
2,4801789,AM Dirtworks & Aggregate Sales,"120 Dally Ln, Buffalo, WY, 82834",Matt Mitchell
3,4201449,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co
4,4201450,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co


In [28]:
mines_csv.id = mines_csv.id.astype(str)
mines_csv.dtypes

id                 object
operator_name      object
mine_address       object
main_controller    object
dtype: object

In [29]:
mines_csv.to_csv('mines_extended.csv', index=False)

In [30]:
pd.read_csv('mines_extended.csv').shape

(129, 4)

In [31]:
pd.read_csv('mines_extended.csv', converters = {'id': str}).head()

,id,operator_name,mine_address,main_controller
0,3503598,Newberg Rock & Dirt,"6860 Anderson Rd., Aurora, OR, 97002",S-2 Contractors Inc
1,0502030,Allied Dirt Moving Company,"2360 West 2nd Ave, DENVER, CO, 80010",Allied Dirt Moving Company
2,4801789,AM Dirtworks & Aggregate Sales,"120 Dally Ln, Buffalo, WY, 82834",Matt Mitchell
3,4201449,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co
4,4201450,Atlas-Dirty Devil Mining,"GREEN RIVER, UT, 84525",Atlas Resources Inc & Dirty Devil Mining Co
